# WESAD FastGRNN

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
from sklearn.model_selection import train_test_split
import pathlib
from os import mkdir

In [2]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val
def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))
def bagData(X, Y, subinstanceLen, subinstanceStride):
    numClass = 3
    numSteps = 175
    numFeats = 8
    assert X.ndim == 3
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0
    assert subinstanceStride <= numSteps
    assert subinstanceStride >= 0
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

In [4]:
subinstanceLen=88
subinstanceStride=30
extractedDir = '/home/sf/data/WESAD/'
# mkdir('/home/sf/data/WESAD/Fast_GRNN/88_30')
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + 'Fast_GRNN' '/%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

Num train 95450
Num test 26514
Num val 10606


In [7]:
outDir

'/home/sf/data/WESAD/Fast_GRNN/88_30/'

In [8]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# FastGRNN and FastRNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_FastGRNN
from edgeml.graph.rnn import EMI_FastRNN
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [9]:
# Network parameters for our FastGRNN + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 88
NUM_FEATS = 8
FORGET_BIAS = 1.0
NUM_OUTPUT = 3
USE_DROPOUT = False
KEEP_PROB = 0.9

# Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"

# Ranks of Parameter matrices for low-rank parameterisation to compress models.
WRANK = 5
URANK = 6

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 175

# Number of epochs in *one iteration*
NUM_EPOCHS = 3

# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4

# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 6

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn'

# Loading Data

In [10]:
# Loading the data
path='/home/sf/data/WESAD/Fast_GRNN/88_30/'
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_test.shape)
print("y_test shape is:", y_test.shape)

x_train shape is: (95450, 4, 88, 8)
y_train shape is: (95450, 4, 3)
x_test shape is: (26514, 4, 88, 8)
y_test shape is: (26514, 4, 3)


# Computation Graph

In [11]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict

    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
if USE_DROPOUT is True:
    EMI_FastGRNN.feedDictFunc = feedDictFunc

In [12]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiFastGRNN = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                           gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

In [13]:
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (95450, 4, 88, 8)
y_train shape is: (95450, 4, 3)
x_test shape is: (10606, 4, 88, 8)
y_test shape is: (10606, 4, 3)


In [14]:
tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    # Obtain the iterators to each batch of the data
    x_batch, y_batch = inputPipeline()
    # Create the forward computation graph based on the iterators
    y_cap = emiFastGRNN(x_batch)
    # Create loss graphs and training routines
    emiTrainer(y_cap, y_batch)

# EMI Driver

In [15]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiFastGRNN, emiTrainer)

emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

Update policy: top-k
Training with MI-RNN loss for 3 rounds
Round: 0
Epoch   2 Batch   543 ( 1635) Loss 0.00083 Acc 0.98429 | Val acc 0.97662 | Model saved to /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn, global_step 1000
Epoch   2 Batch   543 ( 1635) Loss 0.00042 Acc 0.99143 | Val acc 0.98982 | Model saved to /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn, global_step 1001
Epoch   2 Batch   543 ( 1635) Loss 0.00036 Acc 0.99286 | Val acc 0.99133 | Model saved to /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn, global_step 1002
Epoch   2 Batch   543 ( 1635) Loss 0.00033 Acc 0.99429 | Val acc 0.99274 | Model saved to /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn, global_step 1003
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1003
Round: 1
Epoch   2 Batch   543 ( 1635) Loss 0.00027 Acc 0.99429 | Val acc 0.99481 | Model saved to /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn, global_step 1004
Epoch

In [16]:
# Early Prediction Policy: We make an early prediction based on the predicted classes
#     probability. If the predicted class probability > minProb at some step, we make
#     a prediction at that step.
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings

In [17]:
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))

Accuracy at k = 2: 0.998567
Additional savings: 0.960761


In [18]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.998831   0.998532   0.998504   0.998561   0.998831   0.998831   
1    2  0.998567   0.998295   0.998517   0.998074   0.998567   0.998567   
2    3  0.997850   0.997675   0.998287   0.997069   0.997850   0.997850   
3    4  0.996040   0.995706   0.997310   0.994133   0.996040   0.996040   

   micro-rec  
0   0.998831  
1   0.998567  
2   0.997850  
3   0.996040  
Max accuracy 0.998831 at subsequencelength 1
Max micro-f 0.998831 at subsequencelength 1
Micro-precision 0.998831 at subsequencelength 1
Micro-recall 0.998831 at subsequencelength 1
Max macro-f 0.998532 at subsequencelength 1
macro-precision 0.998504 at subsequencelength 1
macro-recall 0.998561 at subsequencelength 1


## Picking the best model

In [19]:
devnull = open(os.devnull, 'r')
for val in modelStats:
    round_, acc, modelPrefix, globalStep = val
    emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
 
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
    print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
    print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 

INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1003
Round:  0, Validation accuracy: 0.9927, Test Accuracy (k = 2): 0.960361, Additional savings: 0.372858
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1007
Round:  1, Validation accuracy: 0.9973, Test Accuracy (k = 2): 0.926303, Additional savings: 0.508829
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1011
Round:  2, Validation accuracy: 0.9964, Test Accuracy (k = 2): 0.950743, Additional savings: 0.585428
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1015
Round:  3, Validation accuracy: 0.9972, Test Accuracy (k = 2): 0.997435, Additional savings: 0.945828
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1018
Round:  4, Validation accuracy: 0.9985, Test Accuracy (k = 2): 0.998491, Additiona

In [20]:
dataset="WESAD"
model="fast-grnn"
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 700, #subinstance length.
    "NUM_FEATS" : 8,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 3,
    "USE_DROPOUT" : 0, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.9,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "WRANK" : 5,
    "URANK" : 6,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 175,
    "NUM_EPOCHS" : 3,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 4,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

fast_dict = {**params}
fast_dict["k"] = k
fast_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
fast_dict["additional_savings"] = getEarlySaving(predictionStep, NUM_TIMESTEPS)
fast_dict["y_test"] = BAG_TEST
fast_dict["y_pred"] = bagPredictions

In [21]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
print (tabulate(df, headers=list(df.columns), tablefmt='grid'))

dirname = "/home/sf/data/WESAD/Fast_GRNN/"
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(fast_dict,open(dirname + "/fast_dict_" + filename + ".pkl",mode='wb'))

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.998831   0.998532   0.998504   0.998561   0.998831   0.998831   
1    2  0.998567   0.998295   0.998517   0.998074   0.998567   0.998567   
2    3  0.997850   0.997675   0.998287   0.997069   0.997850   0.997850   
3    4  0.996040   0.995706   0.997310   0.994133   0.996040   0.996040   

   micro-rec  
0   0.998831  
1   0.998567  
2   0.997850  
3   0.996040  
Max accuracy 0.998831 at subsequencelength 1
Max micro-f 0.998831 at subsequencelength 1
Micro-precision 0.998831 at subsequencelength 1
Micro-recall 0.998831 at subsequencelength 1
Max macro-f 0.998532 at subsequencelength 1
macro-precision 0.998504 at subsequencelength 1
macro-recall 0.998561 at subsequencelength 1
+----+-------+----------+-------------+-------------+-------------+-------------+-------------+-------------+
|    |   len |      acc |   macro-fsc |   macro-pre |   macro-rec |   micro-fsc |   micro-pre |   micro-rec |
+====+====